**The PTB Diagnostic ECG Database**
* **Number of Samples:** 14552
* **Number of Categories:** 2
* **Sampling Frequency:** 125Hz
* **Data Source:** Physionet's PTB Diagnostic Database
* **Remark:** All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188.

The final element of the row is the dependant variable denotes the class that sample belongs to

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
dfs=[pd.read_csv('../input/heartbeat/ptbdb_'+x+'.csv') for x in ['normal','abnormal'] ]

In [4]:
dfs[0]

In [5]:
dfs[1]

In [6]:
for df in dfs:
    df.columns=list(range(len(df.columns)))  #changing the column names from 1 to len

In [7]:
dfs[0]

In [8]:
data = pd.concat(dfs,axis=0).sample(frac=1.0,random_state=1).reset_index(drop=True)
#concat the dfs list #up and down
#shuffle the data
#reset the index and drop the original index

In [9]:
data.head()

In [10]:
#Data Preprocessing #Data already scaled
x=data.drop(187,axis=1).copy() #independant variables
y=data[187].copy() #dependant variables

In [11]:
x

In [12]:
y

In [13]:
#train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=1)

In [14]:
inputs = tf.keras.Input(shape=(X_train.shape[1],)) #187 columns as inputs

#2 Dense layer 64 neurons 
x= tf.keras.layers.Dense(64,activation='relu')(inputs)
x= tf.keras.layers.Dense(64,activation='relu')(x)

#Ouputs is one neuron #sigmoid 0-1 range
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(x)

In [15]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [16]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
 
)

In [17]:
history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[ #to avoid overfitting
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [18]:
results=model.evaluate(X_test,y_test) #loss,accuracy,AUC

In [19]:
print('Test Accuracy: {:2f}%'.format(results[1]*100))
print('     Test AUC: {:4f}'.format(results[2]))
print('    Test loss: {:4f}'.format(results[0]))

**The Neural Network treats every feature independently and hence doesnot learn about the patterns between them, to overcome this we use the Recurrent neural network LSTM**

In [20]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

expand = tf.expand_dims(inputs, axis=2)
gru = tf.keras.layers.GRU(256, return_sequences=True)(expand)
flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model_lstm = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model_lstm.summary())

In [21]:
model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

history = model_lstm.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [22]:
results_lstm = model_lstm.evaluate(X_test, y_test, verbose=0)

In [23]:
print('LSTM-Test Accuracy: {:2f}%'.format(results_lstm[1]*100))
print('     LSTM-Test AUC: {:4f}'.format(results_lstm[2]))
print('     LSTM-Test loss: {:4f}'.format(results_lstm[0])) 
#Better accuracy and better AUC